In [1]:
import streamlit as st
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.document_loaders import YoutubeLoader
import os
from dotenv import load_dotenv
from youtube_transcript_api import YouTubeTranscriptApi
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS


d:\study\Langchain\.lan\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()
gemini_api_key = os.getenv('Gemini_api_key')
ytt_api = YouTubeTranscriptApi()

In [3]:
embedding = GoogleGenerativeAIEmbeddings(model='models/text-embedding-004',
                                         google_api_key=gemini_api_key)

model = ChatGoogleGenerativeAI(model='gemini-2.0-flash',
                               api_key=gemini_api_key)

In [4]:
video_id = 'o50N3-OaGdM'
result = ytt_api.fetch(video_id,languages=['en'])
transcript = "".join(chunk.text for chunk in result)


In [10]:
transcript

"But leopards are the most versatile ofall the big cats. Adept at finding coverin the most unpromising[Music][Music]places. The steep walls of the gully arenow her cover.for anambush. The male puku is close enough,but he's too big totackle. She needs to slip past himwithout being seen.If he spots her, he'll blow her cover.[Music]Slowly doesit. To succeed here, she needs to findprey grazing close to theedge or better still in the gullyitself.Frustration. Success would have savedoff hunger for a week.[Music]But while there's prey around, there's[Music]hope. Peeking over the top is a risk,but it's the quickest way to find a newtarget.[Music][Music][Music][Applause]A burst of speed of 65 kilometers anhour and it's all over in less than 6seconds. Except it[Music]isn't. Dazed and disorientated, theimpala makes a miraculous escape.Somewhere along this seemingly barrenstretch of sand, there isfood in great[Music]quantity. Capefurs. There are around 10,000 of themhere.Adult seals are large ands

In [5]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                          chunk_overlap=200
                                        )


In [26]:
chunk = splitter.split_text(transcript)
chunk


["But leopards are the most versatile ofall the big cats. Adept at finding coverin the most unpromising[Music][Music]places. The steep walls of the gully arenow her cover.for anambush. The male puku is close enough,but he's too big totackle. She needs to slip past himwithout being seen.If he spots her, he'll blow her cover.[Music]Slowly doesit. To succeed here, she needs to findprey grazing close to theedge or better still in the gullyitself.Frustration. Success would have savedoff hunger for a week.[Music]But while there's prey around, there's[Music]hope. Peeking over the top is a risk,but it's the quickest way to find a newtarget.[Music][Music][Music][Applause]A burst of speed of 65 kilometers anhour and it's all over in less than 6seconds. Except it[Music]isn't. Dazed and disorientated, theimpala makes a miraculous escape.Somewhere along this seemingly barrenstretch of sand, there isfood in great[Music]quantity. Capefurs. There are around 10,000 of themhere.Adult seals are large",
 

In [28]:
vector_store = FAISS.from_texts(chunk,embedding)

In [29]:
retriever = vector_store.as_retriever(search_type = 'similarity', search_kwargs={"k":4})


In [30]:
res =retriever.invoke("give me some details about leopard")


In [31]:
res

[Document(id='ac1204c5-742f-4647-9468-277d3edfad26', metadata={}, page_content="But leopards are the most versatile ofall the big cats. Adept at finding coverin the most unpromising[Music][Music]places. The steep walls of the gully arenow her cover.for anambush. The male puku is close enough,but he's too big totackle. She needs to slip past himwithout being seen.If he spots her, he'll blow her cover.[Music]Slowly doesit. To succeed here, she needs to findprey grazing close to theedge or better still in the gullyitself.Frustration. Success would have savedoff hunger for a week.[Music]But while there's prey around, there's[Music]hope. Peeking over the top is a risk,but it's the quickest way to find a newtarget.[Music][Music][Music][Applause]A burst of speed of 65 kilometers anhour and it's all over in less than 6seconds. Except it[Music]isn't. Dazed and disorientated, theimpala makes a miraculous escape.Somewhere along this seemingly barrenstretch of sand, there isfood in great[Music]qua

In [32]:
for r in res:
    print(r.page_content)

But leopards are the most versatile ofall the big cats. Adept at finding coverin the most unpromising[Music][Music]places. The steep walls of the gully arenow her cover.for anambush. The male puku is close enough,but he's too big totackle. She needs to slip past himwithout being seen.If he spots her, he'll blow her cover.[Music]Slowly doesit. To succeed here, she needs to findprey grazing close to theedge or better still in the gullyitself.Frustration. Success would have savedoff hunger for a week.[Music]But while there's prey around, there's[Music]hope. Peeking over the top is a risk,but it's the quickest way to find a newtarget.[Music][Music][Music][Applause]A burst of speed of 65 kilometers anhour and it's all over in less than 6seconds. Except it[Music]isn't. Dazed and disorientated, theimpala makes a miraculous escape.Somewhere along this seemingly barrenstretch of sand, there isfood in great[Music]quantity. Capefurs. There are around 10,000 of themhere.Adult seals are large
Its e

## Chain


In [42]:
from langchain_core.runnables import RunnableParallel,RunnablePassthrough,RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [45]:
def format_docs(retrieve_docs):
    context_text = "\n\n".join(doc.page_content for doc in retrieve_docs)
    return context_text

In [46]:
parellel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question' : RunnablePassthrough()
})

In [50]:
parellel_chain.invoke("can you summerize video")

{'context': "for one so[Music]small. He's really starting to panicnow.His mother isinexperienced. She's only just realizedher[Music]mistake. Both will have learned fromwhat[Music]happened. He won't misbehave while thefamily are traveling again.[Music]\n\nof the[Music]herd. The elephants are heading to the[Music]swamp, a place to cool off as the dayheats up.But the mud can be treacherous andbabies must only enter with theirmums. But this little one seems to haveforgotten therules. Chasing egrets isirresistible. But as the herd enter theswamp, the baby is being left behind.Realizing his predicament, he looks forhis mother and thinks he's foundher. He tags along as the group headinto theswamp, but he's following the wrong[Applause]elephant. With his mother nowhere to beseen, he's lost in a confusion of trunksandlegs. The elephants head into the swampwithout him.The baby is now alone and afraid[Music]while his herd appear to have forgottenhim. He can smell that his mother is outthere somew

In [52]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(template='You are chatbot assistant, who answer question from the given context \n {context} \n question:{question}',
                        input_variables=['context','question'])


In [53]:
chain = parellel_chain | prompt | model | StrOutputParser()

In [54]:
chain.invoke('can you summerize the video')

"Here is a summary of the video:\n\nA baby elephant gets separated from its mother and the herd as they head into a swamp. The baby is distracted by egrets and doesn't follow the rules of staying close to its mother. Realizing he's lost, he panics and wanders into the swamp alone, which is dangerous. The mother, being inexperienced, realizes her mistake. The video also shows how baby elephants learn the rules of behavior, like playing nicely with siblings and staying close to their mothers when the family is on the move. Spy cameras disguised as animals, including a dung ball and an egret, are used to capture the elephants' behavior. The video also shows meerkats taking sentry duty to protect their pups from predators. They even bravely face a spy cobra, reacting as they would to a real threat, and are eventually joined by the rest of the meerkat colony."